In [29]:
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
from sagemaker.inputs import TrainingInput
import pandas as pd
import boto3
import sagemaker
from sagemaker.inputs import TrainingInput
import os

In [30]:
bucket_name = 'mybucket228394' 
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-1


In [31]:
try:
    s3 = boto3.client('s3', region_name=my_region)
    
    if my_region == 'us-east-1':
        print(f'Creating bucket {bucket_name} in region {my_region} without LocationConstraint.')
        s3.create_bucket(Bucket=bucket_name)
    else:
        print(f'Creating bucket {bucket_name} in region {my_region} with LocationConstraint {my_region}.')
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': my_region}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)


Creating bucket mybucket228394 in region us-east-1 without LocationConstraint.
S3 bucket created successfully


In [32]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://mybucket228394/xgboost-as-a-built-in-algo/output


# Downloading The Dataset And Storing in S3

In [33]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [34]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [35]:
from sagemaker.inputs import TrainingInput
import pandas as pd
import boto3
import sagemaker
from sagemaker.inputs import TrainingInput
import os

# Save train data to CSV
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

# Upload CSV to S3
s3 = boto3.Session().resource('s3')
s3.Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

# Create S3 input for SageMaker
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
print("Done")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Done


# Building Models Xgboot- Inbuilt Algorithm

In [36]:
# Save test data to CSV
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

# Upload CSV to S3
s3 = boto3.Session().resource('s3')
s3.Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

# Create S3 input for SageMaker
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')
print("Done")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Done


In [37]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')
     

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [38]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }
print("Done")

Done


In [39]:
import sagemaker
from sagemaker.estimator import Estimator

# Construct a SageMaker estimator that calls the xgboost-container
estimator = Estimator(
    image_uri=container,  # Use image_uri instead of image_name
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,  # train_instance_count is now instance_count
    instance_type='ml.m5.2xlarge',  # train_instance_type is now instance_type
    volume_size=5,  # train_volume_size is now volume_size
    output_path=output_path,
    use_spot_instances=True,  # train_use_spot_instances is now use_spot_instances
    max_run=300,  # train_max_run is now max_run
    max_wait=600  # train_max_wait is now max_wait
)
print("Done")

Done


In [40]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-25-11-03-00-261


2024-07-25 11:03:00 Starting - Starting the training job...
2024-07-25 11:03:14 Starting - Preparing the instances for training...
2024-07-25 11:03:59 Downloading - Downloading the training image......
2024-07-25 11:05:00 Training - Training image download completed. Training in progress.
2024-07-25 11:05:00 Uploading - Uploading generated training model[2024-07-25 11:04:52.583 ip-10-2-74-178.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determ

# Deploy Machine Learning Model As Endpoints

In [41]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-07-25-11-06-16-702
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-07-25-11-06-16-702
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-07-25-11-06-16-702


-------!

# Prediction of the Test Data

In [43]:
from sagemaker.serializers import CSVSerializer
import numpy as np

# Load the data into an array
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Set up the CSV serializer
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()

# Perform prediction
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')

# Convert predictions to array
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)


(12357,)


In [44]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [45]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting The Endpoints

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()